In [1]:
import os
from datetime import date, datetime
from zoneinfo import ZoneInfo
from typing import List, Dict, Tuple, Optional, Union
from decimal import Decimal

import django
# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependency of django_ledger...
import pandas as pd

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'dev_env.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
os.chdir('../')

django.setup()

from django_ledger.models.entity import EntityModel
from django.contrib.auth import get_user_model
from django_ledger.io import roles

# Get Your Entity Administrator UserModel

In [2]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    user_model = UserModel.objects.get(username__exact=MY_USERNAME)
except:
    user_model = UserModel(username=MY_USERNAME)
    user_model.set_password(MY_PASSWORD)
    user_model.save()

# Get or Create an Entity Model

In [3]:
from django_ledger.models.entity import EntityModel

ENTITY_NAME = 'One Big Company, LLC'

entity_model = EntityModel.create_entity(
    name=ENTITY_NAME,
    admin=user_model,
    use_accrual_method=True,
    fy_start_month=1
)

entity_model

<EntityModel: EntityModel one-big-company-llc-bxoqd5wy: One Big Company, LLC>

# Chart of Accounts

## Create a Default Chart of Accounts
- Newly created EntityModel do not have a default Code of Accounts yet.

In [4]:
entity_model.has_default_coa()

False

In [5]:
default_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=True,
    commit=True,
    coa_name='My QuickStart CoA'
)

default_coa_model

<ChartOfAccountModel: My QuickStart CoA (coa-qd5wy-efj64qrkll39u8j)>

In [6]:
default_coa_model

<ChartOfAccountModel: My QuickStart CoA (coa-qd5wy-efj64qrkll39u8j)>

In [7]:
entity_model.default_coa == default_coa_model

True

# Populate Entity with Random Data (Optional)

### Define a Start Date for Transactions

In [8]:
START_DTTM = datetime(year=2022, month=10, day=1, tzinfo=ZoneInfo('UTC'))

- This action will populate the EntityModel with random data.
- It will populate a Code of Accounts using a default pre-defined list.
- This approach is for illustration, educational and testing purposes, not encouraged for new production entities.

In [9]:
entity_model.populate_random_data(start_date=START_DTTM)

### EntityModel has now a Default Chart of Accounts

In [10]:
entity_model.has_default_coa()

True

In [11]:
default_coa_model = entity_model.get_default_coa()
default_coa_model

<ChartOfAccountModel: My QuickStart CoA (coa-qd5wy-efj64qrkll39u8j)>

# Chart of Accounts (CoA)
- A Chart of Accounts is a user-defined list of accounts. 
- Each Entity Model must have at least one default Chart of Accounts.

## Django Ledger support multiple chart of accounts.

In [12]:
another_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=False,
    commit=True,
    coa_name='My Empty Chart of Accounts'
)

In [13]:
another_coa_model

<ChartOfAccountModel: My Empty Chart of Accounts (coa-qd5wy-gm20uicgnbmss8u)>

# Accounts

## Default CoA Accounts

In [14]:
default_coa_accounts_qs = entity_model.get_default_coa_accounts()
pd.DataFrame(default_coa_accounts_qs)

,0
0,ASSETS - 1010: Cash (ASSET_CA_CASH/debit)
1,ASSETS - 1050: Short Term Investments (ASSET_C...
2,ASSETS - 1100: Accounts Receivable (ASSET_CA_R...
3,ASSETS - 1110: Uncollectibles (ASSET_CA_UNCOLL...
4,ASSETS - 1200: Inventory (ASSET_CA_INV/debit)
...,...
83,EQUITY - 6294: Garbage (EX_REGULAR/debit)
84,EQUITY - 6295: Electricity (EX_REGULAR/debit)
85,EQUITY - 6300: Property Management (EX_REGULAR...
86,EQUITY - 6400: Vacancy (EX_REGULAR/debit)


## Get CoA Accounts by CoA Model

In [15]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=default_coa_model)
pd.DataFrame(coa_accounts_by_coa_model_qs)

,0
0,ASSETS - 1010: Cash (ASSET_CA_CASH/debit)
1,ASSETS - 1050: Short Term Investments (ASSET_C...
2,ASSETS - 1100: Accounts Receivable (ASSET_CA_R...
3,ASSETS - 1110: Uncollectibles (ASSET_CA_UNCOLL...
4,ASSETS - 1200: Inventory (ASSET_CA_INV/debit)
...,...
83,EQUITY - 6294: Garbage (EX_REGULAR/debit)
84,EQUITY - 6295: Electricity (EX_REGULAR/debit)
85,EQUITY - 6300: Property Management (EX_REGULAR...
86,EQUITY - 6400: Vacancy (EX_REGULAR/debit)


No Accounts yet on this CoA...

In [16]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=another_coa_model)
pd.DataFrame(coa_accounts_by_coa_model_qs)

""


## Get CoA Accounts by CoA Model UUID
- May pass UUID instance instead of ChartOF AccountsModel...

In [17]:
# coa_accounts_by_coa_uuid_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.uuid)
# pd.DataFrame(coa_accounts_by_coa_uuid_qs)

## Get CoA Accounts by CoA Model Slug
- If string is passed, will lookup by slug...

In [18]:
# coa_accounts_by_coa_slug_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.slug)
# pd.DataFrame(coa_accounts_by_coa_slug_qs)

## Get Accounts With Codes and CoA Model
- Assumes default CoA if no coa_model is passed...

In [19]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(code_list=['1010', '1050'])
pd.DataFrame(coa_accounts_by_codes_qs)

,0
0,ASSETS - 1010: Cash (ASSET_CA_CASH/debit)
1,ASSETS - 1050: Short Term Investments (ASSET_C...


Empty ChartOfAccountModel...

In [20]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(
    code_list=['1010', '1050'], 
    coa_model=another_coa_model
)
pd.DataFrame(coa_accounts_by_codes_qs)

""


### Get All Accounts at Once

In [21]:
coa_qs, coa_map = entity_model.get_all_coa_accounts()

A dictionary, CoA Model -> Account List.

In [22]:
coa_map

{<ChartOfAccountModel: My Empty Chart of Accounts (coa-qd5wy-gm20uicgnbmss8u)>: <AccountModelQuerySet []>,
 <ChartOfAccountModel: My QuickStart CoA (coa-qd5wy-efj64qrkll39u8j)>: <AccountModelQuerySet [<AccountModel: ASSETS - 1010: Cash (ASSET_CA_CASH/debit)>, <AccountModel: ASSETS - 1050: Short Term Investments (ASSET_CA_MKT_SEC/debit)>, <AccountModel: ASSETS - 1100: Accounts Receivable (ASSET_CA_RECV/debit)>, <AccountModel: ASSETS - 1110: Uncollectibles (ASSET_CA_UNCOLL/credit)>, <AccountModel: ASSETS - 1200: Inventory (ASSET_CA_INV/debit)>, <AccountModel: ASSETS - 1300: Prepaid Expenses (ASSET_CA_PREPAID/debit)>, <AccountModel: ASSETS - 1510: Notes Receivable (ASSET_LTI_NOTES/debit)>, <AccountModel: ASSETS - 1520: Land (ASSET_LTI_LAND/debit)>, <AccountModel: ASSETS - 1530: Securities (ASSET_LTI_SEC/debit)>, <AccountModel: ASSETS - 1610: Buildings (ASSET_PPE_BUILD/debit)>, <AccountModel: ASSETS - 1611: Less: Buildings Accumulated Depreciation (ASSET_PPE_BUILD_ACCUM_DEPR/credit)>, <Acc

In [23]:
pd.DataFrame(coa_map[default_coa_model])

,0
0,ASSETS - 1010: Cash (ASSET_CA_CASH/debit)
1,ASSETS - 1050: Short Term Investments (ASSET_C...
2,ASSETS - 1100: Accounts Receivable (ASSET_CA_R...
3,ASSETS - 1110: Uncollectibles (ASSET_CA_UNCOLL...
4,ASSETS - 1200: Inventory (ASSET_CA_INV/debit)
...,...
83,EQUITY - 6294: Garbage (EX_REGULAR/debit)
84,EQUITY - 6295: Electricity (EX_REGULAR/debit)
85,EQUITY - 6300: Property Management (EX_REGULAR...
86,EQUITY - 6400: Vacancy (EX_REGULAR/debit)


In [24]:
pd.DataFrame(coa_map[another_coa_model])

""


## Create Account Model
- Creating AccountModel into empty "another_coa_model"...

In [25]:
account_model = entity_model.create_account(
    coa_model=another_coa_model,
    code='1220',
    role=roles.ASSET_CA_INVENTORY,
    name='A new account created from the EntityModel API!',
    balance_type=roles.DEBIT,
    active=True
)

In [26]:
account_model

<AccountModel: ASSETS - 1220: A new account created from the EntityModel API! (ASSET_CA_INV/debit)>

In [27]:
another_coa_accounts_qs = entity_model.get_coa_accounts(coa_model=another_coa_model)
pd.DataFrame(another_coa_accounts_qs)

,0
0,ASSETS - 1220: A new account created from the ...


# Basic Django Ledger Usage
- The LedgerModel name is whatever your heart desires.
- Examples:
    - A month.
    - A customer.
    - A vendor.
    - A project.
- The more ledgers are created, the more segregation and control over transactions is possible.

In [28]:
ledger_model = entity_model.create_ledger(
    name='My October 2023 Ledger', 
    posted=True
)

## Create a Library

In [29]:
from django_ledger.io.io_library import IOLibrary

library = IOLibrary(name='djangocon-2024-library')

## Create and Register a BluePrint

In [30]:
from django_ledger.io.io_library import IOBluePrint

@library.register
def sale_blueprint(
        sale_amount: Union[int, float, Decimal],
        contribution_margin_percent: float,
        description: Optional[str] = None
) -> IOBluePrint:
    blueprint = IOBluePrint()
    cogs_amount = (1 - contribution_margin_percent) * sale_amount
    blueprint.debit(account_code='1010', amount=sale_amount, description=description)
    blueprint.credit(account_code='4010', amount=sale_amount, description=description)
    blueprint.credit(account_code='1200', amount=cogs_amount, description=description)
    blueprint.debit(account_code='5010', amount=cogs_amount, description=description)
    return blueprint

## Get a Cursor

In [31]:
cursor = library.get_cursor(
    entity_model=entity_model, 
    user_model=user_model
)

## Dispatch Instructions

In [32]:
# Option 1 - Use A Ledger Model
cursor.dispatch('sale_blueprint',
                ledger_model=ledger_model,
                sale_amount=34.45,
                contribution_margin_percent=0.13,
                description='Order ID: 123')

# Option 2- Create a New Ledger Model
cursor.dispatch('sale_blueprint',
                ledger_model='ledger-order-id-123',
                sale_amount=90.43,
                contribution_margin_percent=0.17,
                description='Order ID: 123')

## Commit Your Instructions
Not recommended to post both ledger and journal entries. Posted transactions will immediately hit the books.
**result** contains resulting ledger models, journal entries and transactions fro the committed 

In [33]:
stub = cursor.commit(
    post_new_ledgers=True,
    post_journal_entries=True,
    je_timestamp=datetime(2023,12,2,12,10)
    # je_timestamp='2023-12-02 12:10'
)

### Get Financial Statement Report Data for Ledger Model

Balance Sheet

In [34]:
bs_data = ledger_model.digest_balance_sheet(
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

bs_data.get_balance_sheet_data()

{'assets': {'total_balance': Decimal('4.4800000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('0b23c6fa-282f-491c-8bed-ff7726e0acbc'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('34.4500000000000'),
      'balance_abs': Decimal('34.4500000000000')}],
    'total_balance': Decimal('34.4500000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_inv': {'accounts': [{'account_uuid': UUID('6072dfe6-5e0b-44b6-9ed1-0c7beb0ace86'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_inv',
      'code': '1200',
      'name': 'Inventory',
      'balance_typ

Income Statement

In [35]:
is_data = ledger_model.digest_income_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

is_data.get_income_statement_data()

{'operating': {'revenues': [{'account_uuid': UUID('5e35ef10-c45d-42ff-ac93-e416f0bcc317'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('34.4500000000000'),
    'balance_abs': Decimal('34.4500000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('ff8c7ad8-3ba9-4147-aef7-39d6fbad147c'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-29.9700000000000'),
    'balance_abs': Decimal('29.9700000000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [],
 

Cash Flow Statement

In [36]:
cfs_data = ledger_model.digest_cash_flow_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

cfs_data.get_cash_flow_statement_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('4.4800000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': 0},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('29.9700000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('34.4500000000000'),
  'FINANCING': 0,
  'INVESTING': 0},
 'financing': {'GROUP_CFS_FIN_ISSUING_EQUITY': {'desc

All Statements in a Single Call

In [37]:
fin_digest = ledger_model.digest_financial_statements(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

statement_data = fin_digest.get_financial_statements_data()

In [38]:
statement_data['balance_sheet']

{'assets': {'total_balance': Decimal('4.4800000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('0b23c6fa-282f-491c-8bed-ff7726e0acbc'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'op',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('34.4500000000000'),
      'balance_abs': Decimal('34.4500000000000')}],
    'total_balance': Decimal('34.4500000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_inv': {'accounts': [{'account_uuid': UUID('6072dfe6-5e0b-44b6-9ed1-0c7beb0ace86'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'op',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_inv',
      'code': '1200',
      'name': 'Inventory',
      'balance_typ

In [39]:
statement_data['income_statement']

{'operating': {'revenues': [{'account_uuid': UUID('5e35ef10-c45d-42ff-ac93-e416f0bcc317'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': 'op',
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('34.4500000000000'),
    'balance_abs': Decimal('34.4500000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('ff8c7ad8-3ba9-4147-aef7-39d6fbad147c'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': 'op',
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-29.9700000000000'),
    'balance_abs': Decimal('29.9700000000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [],
 

In [40]:
statement_data['cash_flow_statement']

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('4.4800000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': 0},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('29.9700000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('34.4500000000000'),
  'FINANCING': 0,
  'INVESTING': 0},
 'financing': {'GROUP_CFS_FIN_ISSUING_EQUITY': {'desc

# Financial Statement PDF Reports

## Set Up
- Must enable PDF support by installing dependencies via *pipenv*.
    - pipenv install --categories pdf

## Balance Sheet

In [41]:
bs_report = entity_model.get_balance_sheet_statement(
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
bs_data = bs_report.get_report_data()

In [42]:
bs_data

{'assets': {'total_balance': Decimal('61031.44000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('0b23c6fa-282f-491c-8bed-ff7726e0acbc'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('27763.4100000000'),
      'balance_abs': Decimal('27763.4100000000')}],
    'total_balance': Decimal('27763.4100000000'),
    'role_name': 'Current Asset'},
   'asset_ca_recv': {'accounts': [{'account_uuid': UUID('5538dff4-ba16-4435-9536-cdb4efa443ee'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_recv',
      'code': '1100',
      'name': 'Accounts Receivable',
    

### Balance Sheet Statement Raw Data

In [43]:
bs_report.get_report_data()

{'assets': {'total_balance': Decimal('61031.44000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('0b23c6fa-282f-491c-8bed-ff7726e0acbc'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('27763.4100000000'),
      'balance_abs': Decimal('27763.4100000000')}],
    'total_balance': Decimal('27763.4100000000'),
    'role_name': 'Current Asset'},
   'asset_ca_recv': {'accounts': [{'account_uuid': UUID('5538dff4-ba16-4435-9536-cdb4efa443ee'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_recv',
      'code': '1100',
      'name': 'Accounts Receivable',
    

## Income Statement

In [44]:
ic_report = entity_model.get_income_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

ic_data = ic_report.get_report_data()

### Income Statement Raw Data

In [45]:
ic_data

{'operating': {'revenues': [{'account_uuid': UUID('5e35ef10-c45d-42ff-ac93-e416f0bcc317'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('10806.0700000000'),
    'balance_abs': Decimal('10806.0700000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('ff8c7ad8-3ba9-4147-aef7-39d6fbad147c'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-2606.96000000000'),
    'balance_abs': Decimal('2606.96000000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [{'ac

## Cash Flow Statement

In [46]:
cf_report = entity_model.get_cash_flow_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

cf_data = cf_report.get_report_data()

### Cash Flow Statement Raw Data

In [47]:
cf_data

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('3087.690000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-10806.0700000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('-22461.96000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('7943.7500000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('-22236.590000000000'),
  'FINANCING': Decimal('27763.4

## All Financial Statements Data in a single Call

In [48]:
reports = entity_model.get_financial_statements(
    user_model=user_model,
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

In [49]:
bs_data = reports.balance_sheet_statement.get_report_data()
ic_data = reports.income_statement.get_report_data()
cf_data = reports.cash_flow_statement.get_report_data()

In [50]:
reports.income_statement.get_report_data()

{'operating': {'revenues': [{'account_uuid': UUID('5e35ef10-c45d-42ff-ac93-e416f0bcc317'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('10806.0700000000'),
    'balance_abs': Decimal('10806.0700000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('ff8c7ad8-3ba9-4147-aef7-39d6fbad147c'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-2606.96000000000'),
    'balance_abs': Decimal('2606.96000000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [{'ac

In [51]:
reports.cash_flow_statement.get_report_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('3087.690000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-10806.0700000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('-22461.96000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('7943.7500000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('-22236.590000000000'),
  'FINANCING': Decimal('27763.4

# How To Extend the Ledger Model

In [52]:
from django.db import models
from django_ledger.models import LedgerModel

class EmployeeModel(LedgerModel):
    
    dob = models.DateField()
    salary = models.DecimalField()
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
        
    def process_payroll(self, month):
        raise NotImplementedError()
    
    def send_payroll_report(self, month):
        raise NotImplementedError()

RuntimeError: Model class __main__.EmployeeModel doesn't declare an explicit app_label and isn't in an application in INSTALLED_APPS.